# including development builds in jupyter lite

this work augments our [existing lite build](../xxii/2022-12-21-lite-build.ipynb) by vendoring things that i'm actively working on for demonstration.

https://jupyterlite.readthedocs.io/en/latest/howto/python/wheels.html#adding-pyolite-wheels

In [1]:
    if __name__ == "__main__":
        if "INIT" not in locals():
            %pushd ../..
            INIT = True

        with __import__("importnb").imports("ipynb"):
            import tonyfast.xxii.__lite_build as prior

        %reload_ext doit
        %reload_ext pidgy.extras

    from doit import task_params
    from pathlib import Path
    import os, doit

    TARGET = Path("tonyfast")
    SITE = Path("site", "run")
    PYPI = SITE / "pypi"

/home/tbone/Documents/tonyfast


In [2]:
    @task_params([dict(name="pypi", type=list, default=[
        "https://github.com/deathbeds/midgy",
        "https://github.com/deathbeds/pidgy",
        "https://github.com/deathbeds/importnb"
    ])])
    def task_lite(pypi):
        with __import__("importnb").imports("ipynb"):
            from tonyfast.xxii.__lite_build import set_files_imports
            
        env = os.environ.copy()
        env["SETUPTOOLS_SCM_PRETEND_VERSION"] = "6.6.6"
        hatch = list(filter(lambda x: x.startswith("HATCH"), env))
        for x in hatch:
            env.pop(x)
        wheels = []
        for repo in pypi:
            org, _, name = repo.rpartition("/")
            target = TARGET / name
            yield dict(
                name=F"clone:{repo}",
                actions=[F"git clone --depth 1 {repo} {target}"],
                targets=[HEAD := (target / ".git" / "HEAD")],
                clean=[F"rm -rf {target}"],
                uptodate=[target.exists()]
            )
            pypi = Path("pypi").absolute() # this directory needs to relative to build directory
            wheel = pypi / F"{name}-6.6.6-py3-none-any.whl"
            wheels.append(wheel)
            yield dict(
                name=F"build:{name}",
                actions=[
                    (doit.tools.create_folder, [pypi]),
                    doit.tools.CmdAction(F"python -m build --wheel --outdir {pypi}", cwd=target, env=env)
                ],
                clean=True,
                targets=[wheel],
                uptodate=[False]
            )
        yield dict(
            name="build",
            file_dep=wheels,
            actions=[
                "rm -rf tonyfast/pidgy/lite tonyfast/pidgy/.binder",
                "pip install --pre --upgrade notebook", # development version of notebook for v7
                "jupyter lite build --apps repl --apps notebook --apps lab --contents tonyfast --output-dir site/run",
                # voici is activated here when it works
                (set_files_imports, (Path("site/run/files"),))
            ],
            targets=["site/run/index.html"],
            clean=["rm -rf site/run"]
        )